In [ ]:
from tensorflow.keras import layers, optimizers, losses, metrics, activations, regularizers, callbacks
from keras.models import Model
import numpy as np
import pandas as pd
from tensorflow.keras.layers import LSTM

In [ ]:
path = "/content/drive/Othercomputers/My Laptop/project_code/ECG Project files/"
x_train = np.load(path + 'xtrain.npy')
y_train = np.load(path + 'ytrain.npy', allow_pickle=True)
x_test  = np.load(path + 'xtest.npy')
y_test  = np.load(path + 'ytest.npy', allow_pickle=True)
print(x_train.shape)
x_train = x_train.transpose(0, 2, 1)            # transpose working correctly
x_test  = x_test.transpose(0, 2, 1)
print(x_train.shape)
x_train = x_train.reshape(19601, 12, 1000, 1)   # Add another channel
x_test  = x_test.reshape(2198, 12, 1000, 1)

print("x_train :", x_train.shape)
print("y_train :", y_train.shape)
print("x_test  :", x_test.shape)
print("y_test  :", y_test.shape)
print('Data loaded')

(19601, 1000, 12)
(19601, 12, 1000)
x_train : (19601, 12, 1000, 1)
y_train : (19601,)
x_test  : (2198, 12, 1000, 1)
y_test  : (2198,)
Data loaded


In [ ]:
x_train = x_train[:2000]
x_test = x_test[:500]
y_train = y_train[:2000]
y_test = y_test[:500]

In [ ]:
x_test.shape

(500, 12, 1000, 1)

In [ ]:

from sklearn.preprocessing import MultiLabelBinarizer


# Convert multi-label target labels to one-hot encoded matrix
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)
print("Classes:", mlb.classes_)

Classes: ['CD' 'HYP' 'MI' 'NORM' 'STTC']


In [ ]:
# Main Version
input = layers.Input(shape=(12, 1000, 1))

X = layers.Conv2D(filters=32, kernel_size=(1, 5))(input)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)

convC1 = layers.Conv2D(filters=64, kernel_size=(1, 7))(X)

X = layers.Conv2D(filters=32, kernel_size=(1, 5))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 4), strides=1)(X)

convC2 = layers.Conv2D(filters=64, kernel_size=(1, 6))(convC1)

X = layers.Conv2D(filters=64, kernel_size=(1, 5))(X)
X = layers.BatchNormalization()(X)
X = layers.Add()([convC2, X])           # skip Connection
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)

convE1 = layers.Conv2D(filters=32, kernel_size=(1, 4))(X)

X = layers.Conv2D(filters=64, kernel_size=(1, 3))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 4), strides=1)(X)

convE2 = layers.Conv2D(filters=64, kernel_size=(1, 5))(convE1)

X = layers.Conv2D(filters=64, kernel_size=(1, 3))(X)
X = layers.BatchNormalization()(X)
X = layers.Add()([convE2, X])         # skip Connection
X = layers.ReLU()(X)
X = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(X)
print('Added 5 layers for temporal analysis')

X = layers.Conv2D(filters=64, kernel_size=(12, 1))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.GlobalAveragePooling2D()(X)
print('Added 1 layer for spatial Analysis')

X = layers.Flatten()(X)
print(X.shape)

X = layers.Dense(units=128, kernel_regularizer=regularizers.L2(0.005))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.Dropout(rate=0.1)(X)

X = layers.Dense(units=64, kernel_regularizer=regularizers.L2(0.009))(X)
X = layers.BatchNormalization()(X)
X = layers.ReLU()(X)
X = layers.Dropout(rate=0.15)(X)
print('Added 2 fully connected layers')

output = layers.Dense(5, activation='sigmoid')(X)
model = Model(inputs=input, outputs=output)
print(model.summary())

Added 5 layers for temporal analysis
Added 1 layer for spatial Analysis
(None, 64)
Added 2 fully connected layers
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 12, 1000, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 12, 996, 32)  192         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_16 (BatchN  (None, 12, 996, 32)  128        ['conv2d_20[0][0]']              
 ormalization)                                                               

In [ ]:
# Source: https://keras.io/api/callbacks/
# Source: https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6

early    = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
reducelr = callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3)
callback = [early, reducelr]

model.compile(optimizer = optimizers.Adam(learning_rate=0.005),
              loss = losses.BinaryCrossentropy(),
              metrics = [metrics.BinaryAccuracy(), metrics.AUC(curve='ROC', multi_label=True)])

history = model.fit(x_train, y_train, validation_split=0.12, epochs=20, batch_size=64, callbacks=callback)

Epoch 1/20
28/28 [==============================] - 18s 331ms/step - loss: 1.2254 - binary_accuracy: 0.7661 - auc_2: 0.7121 - val_loss: 0.8834 - val_binary_accuracy: 0.8125 - val_auc_2: 0.7436 - lr: 0.0050
Epoch 2/20
28/28 [==============================] - 8s 299ms/step - loss: 0.5805 - binary_accuracy: 0.8300 - auc_2: 0.8337 - val_loss: 0.6248 - val_binary_accuracy: 0.8075 - val_auc_2: 0.7089 - lr: 0.0050
Epoch 3/20
28/28 [==============================] - 8s 302ms/step - loss: 0.4413 - binary_accuracy: 0.8433 - auc_2: 0.8581 - val_loss: 0.5585 - val_binary_accuracy: 0.7775 - val_auc_2: 0.6651 - lr: 0.0050
Epoch 4/20
28/28 [==============================] - 9s 316ms/step - loss: 0.3905 - binary_accuracy: 0.8549 - auc_2: 0.8740 - val_loss: 0.5570 - val_binary_accuracy: 0.7708 - val_auc_2: 0.5970 - lr: 0.0050
Epoch 5/20
28/28 [==============================] - 9s 317ms/step - loss: 0.3669 - binary_accuracy: 0.8640 - auc_2: 0.8810 - val_loss: 0.5631 - val_binary_accuracy: 0.7517 - val_a

In [ ]:
save_path = "/content/drive/Othercomputers/My Laptop/project_code/ECG Project files/"
model.save(save_path + "ST-CNN-5_final1.h5")

In [ ]:

from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score, accuracy_score, auc


def sklearn_metrics(y_true, y_pred):
    y_bin = np.copy(y_pred)
    y_bin[y_bin >= 0.5] = 1
    y_bin[y_bin < 0.5]  = 0

    # Compute area under precision-Recall curve
    auc_sum = 0
    for i in range(5):
      precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_pred[:,i])
      auc_sum += auc(recall, precision)

    print("Accuracy        : {:.2f}".format(accuracy_score(y_true.flatten(), y_bin.flatten())* 100))
    print("Macro AUC score : {:.2f}".format(roc_auc_score(y_true, y_pred, average='macro') * 100))
    print('AUPRC           : {:.2f}'.format((auc_sum / 5) * 100))
    print("Micro F1 score  : {:.2f}".format(f1_score(y_true, y_bin, average='micro') * 100))


In [ ]:
from tensorflow.keras.models import load_model
model = load_model(r'/content/drive/Othercomputers/My Laptop/project_code/ECG Project files/ST-CNN-5_final1.h5')
y_pred_train = model.predict(x_train)
y_pred_test  = model.predict(x_test)

16/16 [==============================] - 1s 41ms/step


In [ ]:

print("Train")
sklearn_metrics(y_train, y_pred_train)
print("\nTest")
sklearn_metrics(y_test, y_pred_test)


Train
Accuracy        : 82.77
Macro AUC score : 91.06
AUPRC           : 73.36
Micro F1 score  : 60.18

Test
Accuracy        : 81.60
Macro AUC score : 86.17
AUPRC           : 64.94
Micro F1 score  : 56.36


In [ ]:
model_ppr = load_model(r'/content/drive/Othercomputers/My Laptop/project_code/ECG Project files/ST-CNN-GAP-5.h5')
y_pred_train1 = model_ppr.predict(x_train)
y_pred_test1  = model_ppr.predict(x_test)

16/16 [==============================] - 1s 43ms/step


In [ ]:
print("Train")
sklearn_metrics(y_train, y_pred_train1)
print("\nTest")
sklearn_metrics(y_test, y_pred_test1)

Train
Accuracy        : 93.17
Macro AUC score : 97.23
AUPRC           : 91.04
Micro F1 score  : 86.09

Test
Accuracy        : 91.00
Macro AUC score : 93.01
AUPRC           : 82.34
Micro F1 score  : 81.14
